In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Prepare Data

bc = datasets.load_breast_cancer()
X,y = bc.data, bc.target

n_samples, n_features = X.shape

In [3]:
print(n_samples, n_features)

569 30


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [5]:
# scale data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [6]:
# convert to torch tensors

X_train  = torch.from_numpy(X_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
y_train  = torch.from_numpy(y_train.astype(np.float32))
y_test  = torch.from_numpy(y_test.astype(np.float32))

In [14]:
X_train.size()

torch.Size([455])

In [15]:
y_train.size()

torch.Size([455])

In [17]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [18]:
y_train.size()

torch.Size([455, 1])

In [20]:
# Logistic Regression Model

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
    
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

    
model = LogisticRegression(n_features)

In [21]:
# Loss and Optimizer
learning_rate = 0.01
criterion = nn.BCELoss() # Binary Cross Entropy Loss

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [22]:
#training loop

num_epochs = 100

for epoch in range(num_epochs):
    # forward pass
    
    y_predicted = model(X_train)
    loss = criterion (y_predicted, y_train)
    
    # backward Pass
    
    loss.backward()
    
    # updates
    
    optimizer.step()
    
    # zero the gradients
    
    optimizer.zero_grad()
    
    if (epoch+1) % 10 ==0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 0.7219
epoch: 20, loss = 0.5762
epoch: 30, loss = 0.4876
epoch: 40, loss = 0.4283
epoch: 50, loss = 0.3857
epoch: 60, loss = 0.3535
epoch: 70, loss = 0.3282
epoch: 80, loss = 0.3077
epoch: 90, loss = 0.2908
epoch: 100, loss = 0.2764
accuracy = 0.9561
